# Alberi di decisione con ID3

In questo notebook ho deciso di implementare l'algoritmo **ID3** seguendo quanto scritto nelle slide della lezione del 12 ottobre 2020.

# ID3

ID3 è un algoritmo che permette di generare un **albero di decisione**. Nell'ambito del machine learning un albero di decisione è un modello predittivo in cui:

1. ogni *nodo interno* dell'albero rappresenta un **attributo**;
2. ogni *arco* corrisponde ad un possibile **valore** che può essere assunto dal nodo genitore, quindi, dall'attributo;
3. ogni nodo *foglia* assegna una **classificazione**.

Gli alberi di decisione vengono utilizzati in diversi ambiti, in particolare, vengono applicati quando è necessario capire in che modo l'algoritmo è giunto ad una determinata conclusione. Infatti, gli alberi di decisione possono essere tradotti in una serie di *condizioni* `if`. In questo modo, l'uomo è in grado di comprendere il percorso (nell'albero) che ha fatto l'algoritmo.

# Implementazione

Per poter stampare l'albero di decisione prodotto da ID3 ho utilizzato il pacchetto `anytree`. Non ho trovato dei pacchetti diversi che soddisfacessero le mie esigenze.

In [2]:
!pip install anytree

     |████████████████████████████████| 51kB 3.8MB/s 


## Step 1

Creazione del nodo radice per l'albero di decisione finale.

In [ ]:
import numpy as np
from math import log
from anytree import Node

class ID3:

    # Step 1: create the root node
    T = Node("Root")

    def __init__(self, S, A):
      self.algorithm(S, A, self.T)

## Step 2

Se gli esempi nell'insieme $S$ sono tutti della stessa classe $c$, ritorna l'albero $T$ etichettato con la classe $c$.

In [ ]:
# Check if all the elements of a set are of the same class c
def areAllElementOfSetEqual(self, S):
    c = S[0]["Sport"]
    for i in range(1, self.cardinality(S)):
        if S[i]["Sport"] != c:
            return ""
    return c

## Step 3

Se l'insieme degli attributi $A$ è vuoto, ritorna l'albero $T$ etichettato con la classe di maggiornaza in $S$.

In [ ]:
# Determine the majority class in S
def majorityClassOfSet(self, S, A):
    classes = {}

    for s in S:
        if s["Sport"] not in classes:
            classes[s["Sport"]] = 1
        else:
            classes[s["Sport"]] += 1

    return A[int(np.argmax(classes))]

## Step 4

Si scelga $a \in A$ tale che $a$ sia l'attributo **ottimo** in $A$.

In [ ]:
# Determine the optimal attribute in A
def optimalAttribute(self, S, A):

    # If the |A| = 1, consider the only one attribute in A as optimal
    if self.cardinality(A) == 1:
        return A[0]

    information_gains = []

    for a in A:
        information_gains.append(self.informationGain(S, a))

    # Determine which attribute has the highest Information Gain
    index = np.argmax(information_gains)

    return A[int(index)]

### Information Gain

Per selezionare l'attributo ottimo $a \in A$, si scelga l'attributo che massimizza l'Information Gain $G(S, a)$. L'Information Gain rappresenta la riduzione *attesa* di entropia che si ottiene partizionando gli esempi in $S$ usando l'attributo $a$.

La formula per calcolare l'**Information Gain** è: $G(S, a) = E(S) - \sum_{v \in V(a)} \frac{\left|S_{a=v}\right|}{\left|S\right|} \cdot E(S_{a=v})$, dove $a \in A$, $E$ è l'**entropia** e $S_{a=v}$ è l'insieme degli esempi in $S$ con attributo $a$ e valore dell'attributo uguale a $v$.

In [ ]:
# Calculate the Information Gain
def informationGain(self, S, x):
    values = {}
    summation = 0

    for s in S:
        if s[x] not in values:
            values[s[x]] = 1
        else:
            values[s[x]] += 1

    for v in values:
        # Get the examples from S by the value v of the attribute x
        s_x = self.examplesByAttribute(S, x, v)

        summation += (values[v] / self.cardinality(S)) * self.entropy(s_x, method="cross-entropy")

    return self.entropy(S, method="cross-entropy") - summation

### Entropia

L'entropia è una misura del grado di impurità di un insieme di esempi $S$. Siano $C$ il numero di classi, $S_c$ il sottoinsieme di $S$ di esempi di classi $c$ e $p_c = \frac{\left|S_c\right|}{\left|S\right|}$.

I criteri per il calcolo dell'entropia implementati sono:

1. Cross-Entropy: $- \sum_{c=1}^m p_c \cdot \log_2(p_c)$
2. Gini Index: $1 - \sum_{c=1}^m p_c^2$

In [ ]:
# Calculate the entropy
def entropy(self, S, method):
    if method == "cross-entropy":
        return self.crossEntropy(S)
    if method == "gini-index":
        return self.giniIndex(S)

# Calculate the Cross-Entropy
def crossEntropy(self, S):
    classes = {}

    for s in S:
        if s["Sport"] not in classes:
            classes[s["Sport"]] = 1
        else:
            classes[s["Sport"]] += 1

    E = 0
    for c in classes:
        p_c = classes[c] / self.cardinality(S)
        E += p_c * log(p_c, 2)

    return -E

# Calculate the Gini Index
def giniIndex(self, S):
    classes = {}

    for s in S:
        if s["Sport"] not in classes:
            classes[s["Sport"]] = 1
        else:
            classes[s["Sport"]] += 1
    
    GI = 0
    for c in classes:
      p_c = classes[c] / self.cardinality(S)
      GI += p_c * p_c

    return 1 - GI

## Step 5 e 6

### Step 5

Si partizioni l'insieme $S$ secondo i possibili valori che l'attributo ottimo $a$ può assumere, ovvero, $S_{a=v_1},\dots,S_{a=v_m}$, dove $m$ è il numero dei possibili valori assunto da $a$.

### Step 6

Siano:
1. $S_j^{'} = S_{a=v_j}$
2. $A^{'} = A - a$

con $a \in A$ tale che $a$ sia l'attributo ottimo in $A$.

Si effettui una chiamata ricorsiva ID3($S_j^{'}$, $A^{'}$) per ogni $j$.



In [ ]:
# Partition the set S by the value v that an attribute x can assume in S
def partition(self, S, x, v):
    partitions = []

    for i in range(self.cardinality(S)):
        if S[i][x] == v:
            partitions.append(S[i])

    return partitions

In [ ]:
# Get all the values that the optimal attribute a can assume in S
values = self.valuesByAttribute(S, a)

# Update the tree T
T_prime = Node(a, parent=T, value=values)

# Remove the current optimal attribute a from A
A.remove(a)

# Make a recursive call for each value that the optimal attribute a can assume in S
for i in range(len(values)):

    # Step 5: partition the set S according to the possible values that the optimal attribute a can assume
    S_prime = self.partition(S, a, values[i])

    # Step 6: recursive call of ID3
    self.algorithm(S_prime, A, T_prime)

# Esecuzione dell'algoritmo

Gli insiemi $S$ e $A$ contengono rispettivamente gli elementi che sono presenti nelle slide della lezione.

É possibile cambare il criterio per il calcolo dell'entropia modficando il parametro `method` del metodo `entropy`. Il metodo `entropy` viene richiamato per il calcolo dell'Information Gain.

In [9]:
import numpy as np
from math import log
from anytree import Node, RenderTree

class ID3:

    # Step 1: create the root node
    T = Node("Root")

    def __init__(self, S, A):
        self.algorithm(S, A, self.T)

    def algorithm(self, S, A, T):
        # Step 2: if all the examples in S are of the same class c, returns the tree T labeled with class c
        c = self.areAllElementOfSetEqual(S)
        if c != "":
            return Node(c, parent=T)

        # Step 3: if A is empty, returns the tree T labeled with the majority class c in S
        if not A:
            c = self.majorityClassOfSet(S, A)
            return Node(c, parent=T)

        # Step 4: let a belongs to A such that a is optimal in A
        a = self.optimalAttribute(S, A)

        # Get all the values that the optimal attribute a can assume in S
        values = self.valuesByAttribute(S, a)

        # Update the tree T
        T_prime = Node(a, parent=T, value=values)

        # Remove the current optimal attribute a from A
        A.remove(a)

        # Make a recursive call for each value that the optimal attribute a can assume in S
        for i in range(len(values)):

            # Step 5: partition the set S according to the possible values that the optimal attribute a can assume
            S_prime = self.partition(S, a, values[i])

            # Step 6: recursive call of ID3
            self.algorithm(S_prime, A, T_prime)

    # Check if all the elements of a set are of the same class c
    def areAllElementOfSetEqual(self, S):
        c = S[0]["Sport"]
        for i in range(1, self.cardinality(S)):
            if S[i]["Sport"] != c:
                return ""
        return c

    # Determine the majority class in S
    def majorityClassOfSet(self, S, A):
        classes = {}

        for s in S:
            if s["Sport"] not in classes:
                classes[s["Sport"]] = 1
            else:
                classes[s["Sport"]] += 1

        return A[int(np.argmax(classes))]

    # Determine the optimal attribute in A
    def optimalAttribute(self, S, A):

        # If the |A| = 1, consider the only one attribute in A as optimal
        if self.cardinality(A) == 1:
            return A[0]

        information_gains = []

        for a in A:
            information_gains.append(self.informationGain(S, a))

        # Determine which attribute has the highest Information Gain
        index = np.argmax(information_gains)

        return A[int(index)]

    # Get the cardinality of a set
    def cardinality(self, S):
        return len(S)

    # Calculate the Information Gain
    def informationGain(self, S, x):
        values = {}
        summation = 0

        for s in S:
            if s[x] not in values:
                values[s[x]] = 1
            else:
                values[s[x]] += 1

        for v in values:
            # Get the examples from S by the value v of the attribute x
            s_x = self.examplesByAttribute(S, x, v)

            summation += (values[v] / self.cardinality(S)) * self.entropy(s_x, method="cross-entropy")

        return self.entropy(S, method="cross-entropy") - summation

    # Get the examples from the set S with attribute x and value v
    def examplesByAttribute(self, S, x, v):
        s_x = []
        for s in S:
            if s[x] == v:
                s_x.append(s)
        return s_x

    # Calculate the entropy
    def entropy(self, S, method):
        if method == "cross-entropy":
            return self.crossEntropy(S)
        if method == "gini-impurity":
            return self.giniImpurity(S)

    # Calculate the Cross-Entropy
    def crossEntropy(self, S):
        classes = {}

        for s in S:
            if s["Sport"] not in classes:
                classes[s["Sport"]] = 1
            else:
                classes[s["Sport"]] += 1

        E = 0
        for c in classes:
            p_c = classes[c] / self.cardinality(S)
            E += p_c * log(p_c, 2)

        return -E

    # Calculate the Gini Impurity
    def giniImpurity(self, S):
        classes = {}

        for s in S:
            if s["Sport"] not in classes:
                classes[s["Sport"]] = 1
            else:
                classes[s["Sport"]] += 1
        
        GI = 0
        for c in classes:
          p_c = classes[c] / self.cardinality(S)
          GI += p_c * p_c

        return 1 - GI

    # Get the values that an attribute x can assume in S
    def valuesByAttribute(self, S, x):
        values = []

        for i in range(self.cardinality(S)):
            if S[i][x] not in values:
                values.append(S[i][x])

        return values

    # Partition the set S by the value v that an attribute x can assume in S
    def partition(self, S, x, v):
        partitions = []

        for i in range(self.cardinality(S)):
            if S[i][x] == v:
                partitions.append(S[i])

        return partitions

S = [
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Sport": "No"},
    {"Outlook": "Sunny", "Temperature": "Hot", "Humidity": "High", "Wind": "Strong", "Sport": "No"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "High", "Wind": "Weak", "Sport": "Yes"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Sport": "Yes"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Sport": "Yes"},
    {"Outlook": "Rain", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Sport": "No"},
    {"Outlook": "Overcast", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Strong", "Sport": "Yes"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "High", "Wind": "Weak", "Sport": "No"},
    {"Outlook": "Sunny", "Temperature": "Cool", "Humidity": "Normal", "Wind": "Weak", "Sport": "Yes"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Weak", "Sport": "Yes"},
    {"Outlook": "Sunny", "Temperature": "Mild", "Humidity": "Normal", "Wind": "Strong", "Sport": "Yes"},
    {"Outlook": "Overcast", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Sport": "Yes"},
    {"Outlook": "Overcast", "Temperature": "Hot", "Humidity": "Normal", "Wind": "Weak", "Sport": "Yes"},
    {"Outlook": "Rain", "Temperature": "Mild", "Humidity": "High", "Wind": "Strong", "Sport": "No"}
]

A = ['Outlook', 'Temperature', 'Humidity', 'Wind']

decision_tree = ID3(S, A)

# Show the tree produced by ID3
for prefix, filling, node in RenderTree(decision_tree.T):
    print("{}{}".format(prefix, node.name))

Root
└── Outlook
    ├── Humidity
    │   ├── No
    │   └── Yes
    ├── Yes
    └── Wind
        ├── Yes
        └── No
